<a href="https://colab.research.google.com/github/podobongbong/the_project/blob/main/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/podobongbong/the_project.git
import os
os.chdir('/content/the_project')
import pandas as pd


fatal: destination path 'the_project' already exists and is not an empty directory.


In [2]:

# Energy Data (2004-2023)
energy_data = pd.read_csv('energy.csv')
energy_data.columns = ['year','city','district','type','jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

#주택용
energy_data = energy_data[energy_data['type']=='주택용']
monthly_columns = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
aggregated_data = energy_data.groupby(['year','city'])[monthly_columns].sum().reset_index()

energy_data['city'] = energy_data['city'].replace({'서울특별시': 'seoul', '부산광역시': 'busan'})
aggregated_energy_data = energy_data.groupby(['year', 'city'])[monthly_columns].sum().reset_index()

energy_seoul = aggregated_energy_data[aggregated_energy_data['city'] == 'seoul']
energy_busan = aggregated_energy_data[aggregated_energy_data['city'] == 'busan']


In [3]:
# Population Data (2008-2023)
population_data = pd.read_csv('/content/the_project/population.csv')
population_data.columns = ['year', 'city', 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
population_data['city'] = population_data['city'].replace({'seoul': 'seoul', 'busan': 'busan'})


In [4]:

# Cliamte Data (2004-2023)
climate_busan = pd.read_csv('/content/the_project/climate_busan.csv')
climate_seoul = pd.read_csv('/content/the_project/climate_seoul.csv')
climate_data = pd.concat([climate_busan, climate_seoul], ignore_index=True)
df= climate_data.copy()

df['temp_range'] = df['high'] - df['low']
df['high_deviation'] = df['high'] - df['average']
df['low_deviation'] = df['average'] - df['low']
df = df.dropna()

## Climate feature selection: Primary Component Analysis
from sklearn.decomposition import PCA
import pandas as pd
climate_vars = df[['average', 'low', 'high', 'temp_range','high_deviation','low_deviation']]
climate_vars = climate_vars.dropna()

## Apply PCA to reduce to 1 or 2 components
pca = PCA(n_components=1)  # Adjust to 1 or 2 based on analysis
climate_pca = pca.fit_transform(climate_vars)
df['climate_summary'] = climate_pca[:, 0]  # Only the first component
df['city'] = df['station'].replace({108: 'seoul', 159: 'busan'})
df['city'] = df['city'].astype(str)

climate = df[['ym','city','climate_summary']]

In [5]:

e_seoul = energy_seoul[monthly_columns].values.flatten().reshape(-1, 1)
e_busan = energy_seoul[monthly_columns].values.flatten().reshape(-1, 1)

p_seoul = population_data[population_data['city'] == 'seoul'][monthly_columns].values.flatten().reshape(-1, 1)
p_busan = population_data[population_data['city'] == 'busan'][monthly_columns].values.flatten().reshape(-1, 1)


In [6]:
df

,ym,station,average,low,high,temp_range,high_deviation,low_deviation,climate_summary,city
0,\t\t\t2004-01,159.0,2.8,-0.8,7.9,8.7,5.1,3.6,-19.154920,busan
1,\t\t\t2004-02,159.0,6.6,2.6,11.7,9.1,5.1,4.0,-12.843720,busan
2,\t\t\t2004-03,159.0,8.8,4.8,13.8,9.0,5.0,4.0,-9.089009,busan
3,\t\t\t2004-04,159.0,14.0,10.1,19.3,9.2,5.3,3.9,0.122118,busan
4,\t\t\t2004-05,159.0,17.3,14.3,20.9,6.6,3.6,3.0,5.559412,busan
...,...,...,...,...,...,...,...,...,...,...
476,\t\t\t2023-08,108.0,27.2,24.3,30.8,6.5,3.6,2.9,22.748335,seoul
477,\t\t\t2023-09,108.0,23.7,20.2,27.6,7.4,3.9,3.5,16.453904,seoul
478,\t\t\t2023-10,108.0,15.8,11.5,21.0,9.5,5.2,4.3,2.923491,seoul
479,\t\t\t2023-11,108.0,6.8,2.9,11.5,8.6,4.7,3.9,-12.634511,seoul


In [7]:
c_vars = ['ym','average', 'low', 'high', 'temp_range','high_deviation','low_deviation']
seoul = df[df['city']=='seoul'][c_vars]
busan = df[df['city']=='busan'][c_vars]

In [11]:
seoul['energy']=e_seoul
busan['energy']=e_busan

In [66]:
seoul_08 = seoul.iloc[48:240].reset_index(drop=True)
busan_08 = busan.iloc[48:240].reset_index(drop=True)
seoul_08['pop'] = p_seoul[0:192]
busan_08['pop'] = p_seoul[0:192]

In [69]:
# prompt: seoul_08 exploratory data analysis

import matplotlib.pyplot as plt
import seaborn as sns

# Correlation Matrix
correlation_matrix = seoul_08.corr()
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Seoul Energy Consumption and Climate Variables (2008-2017)')
plt.show()


# Scatter Plots
plt.figure(figsize=(10, 6))
plt.scatter(seoul_08['average'], seoul_08['energy'])
plt.xlabel('Average Temperature')
plt.ylabel('Energy Consumption')
plt.title('Energy Consumption vs. Average Temperature (Seoul 2008-2017)')
plt.show()

plt.figure(figsize=(10, 6))
plt.scatter(seoul_08['high'], seoul_08['energy'])
plt.xlabel('High Temperature')
plt.ylabel('Energy Consumption')
plt.title('Energy Consumption vs. High Temperature (Seoul 2008-2017)')
plt.show()

plt.figure(figsize=(10, 6))
plt.scatter(seoul_08['low'], seoul_08['energy'])
plt.xlabel('Low Temperature')
plt.ylabel('Energy Consumption')
plt.title('Energy Consumption vs. Low Temperature (Seoul 2008-2017)')
plt.show()

# Boxplots
plt.figure(figsize=(10, 6))
sns.boxplot(x='ym', y='energy', data=seoul_08)
plt.xlabel('Month')
plt.ylabel('Energy Consumption')
plt.title('Monthly Energy Consumption Distribution (Seoul 2008-2017)')
plt.xticks(rotation=45)
plt.show()


# Time Series Plots
plt.figure(figsize=(12, 6))
plt.plot(seoul_08['ym'], seoul_08['energy'])
plt.xlabel('Month')
plt.ylabel('Energy Consumption')
plt.title('Seoul Energy Consumption Trend (2008-2017)')
plt.xticks(rotation=45)
plt.show()


# Descriptive Statistics
print(seoul_08.describe())

ValueError: could not convert string to float: '\t\t\t2008-01'